In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
'''ADD MORE IF NEEDED'''
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Input, ReLU, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, schedules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [8]:
class ClassName(Model):
    def __init__(self):
        '''
            CHANGE TO APPLY (here 28*28 use for example MNIST)
        '''
        super().__init__()
        self.flatten = Flatten()
        self.dense1 = Dense(units=256, name='dense1')
        self.dense2 = Dense(units=128, name='dense2')
        self.dense3 = Dense(units=64, name='dense3')
        self.dense4 = Dense(units=10, name='dense4')
        self.relu = ReLU()

        '''
            NOT CHANGE HERE
        '''
        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.optimizer= None
        self.step_lr_scheduler = None
        self.loss = None
        self.metric = None
        self.checkpoint = None
        self.earlystop = None

    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dense4(x)
        return x

    def prepare_data(self, percentage_val=0.2):
        mnist = keras.datasets.mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        #train_size = int((1-percentage_val)*(x_train.shape[0]))
        #val_size = int((percentage_val)*(x_train.shape[0]))
        self.train_dataset = (x_train, y_train)
        self.test_dataset = (x_test, y_test)
        print(self.train_dataset[0].shape)
        print(self.test_dataset[0].shape)
        return self.train_dataset, self.test_dataset

    def train_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE, TARGET_SIZE, COLOR_MODE,CLASS_MODE TO APPLY
        '''
        train_gen = ImageDataGenerator(rescale=1./255)
        # Optinal for data augumentation
        # rotation_range=20,
        # horizontal_flip=True,
        # width_shift_range=0.2, height_shift_range=0.2,
        # shear_range=0.2, zoom_range=0.2)
        self.train_dataset = train_gen.flow_from_directory(
           directory=PATH +'/train',
           target_size=(28,28),
           class_mode='sparse',
           shuffle=True,
           color_mode='rbg',
           batch_size=batch_size
        )
        return self.train_dataset

    def val_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE, TARGET_SIZE, COLOR_MODE,CLASS_MODE TO APPLY
        '''
        val_gen = ImageDataGenerator(rescale=1./255)
        # Optinal for data augumentation
        # rotation_range=20,
        # horizontal_flip=True,
        # width_shift_range=0.2, height_shift_range=0.2,
        # shear_range=0.2, zoom_range=0.2)
        self.val_dataset = val_gen.flow_from_directory(
            directory=PATH +'/val',
            target_size=(28,28),
            class_mode='sparse',
            shuffle=False,
            color_mode='rbg',
            batch_size=batch_size
        )
        return self.val_dataset

    def test_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE, TARGET_SIZE, COLOR_MODE,CLASS_MODE TO APPLY
        '''
        test_gen = ImageDataGenerator(rescale=1./255)
        # Optinal for data augumentation
        # rotation_range=20,
        # horizontal_flip=True,
        # width_shift_range=0.2, height_shift_range=0.2,
        # shear_range=0.2, zoom_range=0.2)
        self.test_dataset = test_gen.flow_from_directory(
           directory=PATH +'/test',
           target_size=(28,28),
           class_mode='sparse',
           shuffle=False,
           color_mode='rbg',
           batch_size=batch_size
        )
        return self.test_dataset

    def configure_optimizer(self):
        learning_rate = 0.001
        self.loss = SparseCategoricalCrossentropy(from_logits=True)
        self.optimizer = Adam(learning_rate=learning_rate)
        self.metric = ["accuracy"]
        return  self.loss, self.optimizer, self.metric

    def train_model(self, epochs=100, batch_size=32, verbose=1, validation_split=0.2):
        '''
            CHANGE EPOCHS, BATCH_SIZE, VALIDATE_SPLIT TO APPLY
        '''
        self.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metric)
        history = self.fit(
            x=self.train_dataset[0],
            y=self.train_dataset[1],
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            validation_split=validation_split,
            callbacks=[]
        )
        self.save_model('checkpoint.ckpt')
        return history

    def test_model(self, batch_size):
        self.evaluate(self.test_dataset[0], self.test_dataset[1], batch_size=batch_size, verbose=2)

    def save_model(self, PATH):
        return self.save_weights(PATH)

    def load_model(self, PATH):
        return self.load_weights(PATH)

    def take_checkpoint(self, n_batches, PATH):
        '''
          CHANGE N_BATCHES TO APPLY
        '''
        self.checkpoint = ModelCheckpoint(
            filepath=PATH,
            verbose=1,
            save_weights_only=True,
            save_best_only=False,
            save_freq=5*n_batches
        )
        return self.checkpoint

    def take_earlystop(self, monitor="val_loss",patience=5):
        '''
            CHANGE MONITOR, PATIENCE TO APPLY
        '''
        self.earlystop = EarlyStopping(
            monitor="val_loss",
            patience=patience,
            verbose=2,
            mode='auto'
        )
        return self.earlystop


In [9]:
model = ClassName()
model.prepare_data()
model.configure_optimizer()

(60000, 28, 28)
(10000, 28, 28)


(<keras.src.losses.SparseCategoricalCrossentropy at 0x79f518bdedd0>,
 ListWrapper(['accuracy']))

In [10]:
model.train_model(epochs=5, batch_size=64, verbose=1, validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 9s 11ms/step - loss: 1.5396 - accuracy: 0.8702 - val_loss: 0.4096 - val_accuracy: 0.9222
Epoch 2/5
750/750 [==============================] - 6s 7ms/step - loss: 0.2671 - accuracy: 0.9391 - val_loss: 0.3232 - val_accuracy: 0.9258
Epoch 3/5
750/750 [==============================] - 7s 10ms/step - loss: 0.1729 - accuracy: 0.9552 - val_loss: 0.2403 - val_accuracy: 0.9465
Epoch 4/5
750/750 [==============================] - 5s 7ms/step - loss: 0.1350 - accuracy: 0.9625 - val_loss: 0.1834 - val_accuracy: 0.9542
Epoch 5/5
750/750 [==============================] - 6s 9ms/step - loss: 0.1239 - accuracy: 0.9649 - val_loss: 0.2182 - val_accuracy: 0.9504


In [11]:
model.test_model(batch_size=64)

157/157 - 0s - loss: 0.2408 - accuracy: 0.9480 - 412ms/epoch - 3ms/step
